In [11]:
def read_csv_list(my_list, my_tag):
    tag_name = my_tag
    file_name = tag_name + '_tag_link.csv'
    
    with open(file_name) as file:
        for line in file:
            read_data = file.readline()
            my_list.append(read_data[:-2])
        file.close()

my_tag = 'seoul'
seoul_links = []
read_csv_list(seoul_links, my_tag)
print(seoul_links[0])
print(len(seoul_links))

https://www.instagram.com/p/BsaWrRogf8b/
1712


In [15]:
## 추출하고자 하는 내용은 다음과 같습니다.

## id ( 게시글 아이디 ) -> OK
## username ( 유저 아이디) -> OK
## date ( 게시 날짜 ) -> OK
## contents ( 글 내용 ) -> OK
## tags ( 태그들 ) -> OK
## image_links ( 이미지 링크들 ) -> OK
## like_num ( 좋아요 개수 ) -> OK
## comment_num ( 댓글 개수 ) -> OK

In [16]:
import requests
import re
import time
import json
from bs4 import BeautifulSoup

In [17]:
## 찾아낸 데이터를 JSON 객체로 생성하는 함수를 정의합니다.
def data2json(my_tag, id, username, date, contents, hash_tags, final_image_link, likes_num, comments_num):
    single_data = {
        "find_tag" : my_tag,
        "id" : id,
        "username" : username,
        "date" : date,
        "contents" : contents,
        "hashtags" : hash_tags,
        "imagelinks" : final_image_link,
        "likes" : likes_num,
        "comments" : comments_num
    }
    return single_data

In [23]:
json_list = []

for i in range(len(seoul_links)):
    
    test_url = seoul_links[i]
    req = requests.get(test_url)
    html = req.text
    header = req.headers
    status = req.status_code
    soup = BeautifulSoup(html, 'html.parser')
    script_content = soup.find_all('script')

    ## 게시글 아이디를 링크로부터 가져와 찾아냅니다.
    id_p = re.compile("\/p\/(.*?)\/")
    id = id_p.findall(test_url)[0]

    ## 사용자 이름을 rel='canonical'에서 주소를 가져와 찾아냅니다.
    user_content = soup.find(rel="canonical")
    if user_content:
        username_pp = user_content.get('href')
        username_p = re.compile("instagram.com/(.*)/p")
        username = username_p.findall(str(username_pp))
        if len(username) > 0:
            username = username[0]


    ## 날짜를 header 파일에서 'Date' 부분을 가져와 찾아냅니다.
    date_p = re.compile("uploadDate\":\"(.{19})\"")
    date = date_p.findall(str(script_content))
    if len(date) > 0:
        date = date[0]
    else:
        date = 0
        

    ## 글 내용을 script 내용에서 caption 부분을 가져와 찾아냅니다.
    ## 태그 내용을 contents 에서 빼는 과정이 필요할지도 모릅니다.
    if script_content:   
        contents_p = re.compile("\"caption\":\"(.*?)\"")
        contents = contents_p.findall(str(script_content))
        if len(contents) > 0:
            contents = contents[0]
            

    ## 해쉬태그를 property='instapp:hashtags' 에서 contetn= 이후 부분을 가져와 찾아냅니다.
    meta_content = soup.find_all(property = "instapp:hashtags")
    if meta_content:
        hash_tags_p = re.compile("content=\"(.*?)\"")
        hash_tags = hash_tags_p.findall(str(meta_content))
        

    ## 이미지 링크를 display_resources 에서 가져와 찾아냅니다.
    ## 이미지가 N개일 때, 총 N+1 종류의 이미지 링크가 존재합니다.
    ## 확인해 보니 1번째와 2번째 이미지는 동일하며, 나머지는 서로 다른 이미지였습니다.
    ## 추정해 보면 1번째 이미지는 2번째 이후 이미지들을 대표하는 하나의 이미지라 할 수 있습니다.
    ## 각 이미지에 대해 서로 다른 resolution 으로 3가지의 다른 형태로 저장된 것을 볼 수 있습니다.
    ## 1번째는 640, 2번째는 750, 3번째는 원본 크기인것으로 보이며, 이 중에서 원본 크기의 링크만 저장합니다.
    image_links_p = re.compile("\"display_resources\":\[.*?\]")
    image_links = image_links_p.findall(str(script_content))

    final_image_link = []
    single_image_link_p = re.compile("\"src\":\"(.*?)\"")
    for dif_img in image_links:
        link = single_image_link_p.findall(str(dif_img))[2]
        final_image_link.append(link)
        

    ## 이미지가 1개일 때는 대표 이미지만 존재하므로, 이에 대한 예외처리가 필요하다
    if len(final_image_link) != 1:
        final_image_link = final_image_link[1:]    

    ## like_num, comment_num 을 description 에서 찾아냅니다.
    likes_num_p = re.compile("\"description\":\"(.*?)Likes")
    likes_num = likes_num_p.findall(str(script_content))
    if len(likes_num) > 0:
        likes_num = likes_num[0]
    else:
        likes_num = 0
        
    comments_num_p = re.compile("\"description\":\".*?,(.*?)Comments")
    comments_num = comments_num_p.findall(str(script_content))
    if len(comments_num) > 0:
        comments_num = comments_num[0]
    else:
        comments_num = 0
    

    single_json = data2json(my_tag, id, username, date, contents, hash_tags, final_image_link, likes_num, comments_num)
    json_list.append(single_json)
    time.sleep(0.1)
    if i % 100 == 0:
        print(i, '번째 데이터')
        print(single_json)
        print('------------------')

0 번째 데이터
{'find_tag': 'seoul', 'id': 'BsaWrRogf8b', 'username': 'kathi556g1', 'date': '2019-01-09T11:15:59', 'contents': '#pflege #love #kpop #korea #seoul #bts #saranghae', 'hashtags': ['bts', 'saranghae', 'pflege', 'seoul', 'korea', 'kpop', 'love'], 'imagelinks': ['https://scontent-icn1-1.cdninstagram.com/vp/24d745e0d79638f48c073a6783491f7b/5CBF0F8D/t51.2885-15/e35/47694159_1694374140662642_1495816516982807046_n.jpg?_nc_ht=scontent-icn1-1.cdninstagram.com', 'https://scontent-icn1-1.cdninstagram.com/vp/280fccf8dd001d7b4a555d7393722437/5CD523DB/t51.2885-15/e35/47585725_522155848283529_765483270533959943_n.jpg?_nc_ht=scontent-icn1-1.cdninstagram.com'], 'likes': '9 ', 'comments': ' 1 '}
------------------
100 번째 데이터
{'find_tag': 'seoul', 'id': 'BsaSUWnARJE', 'username': 'mysoonmoo', 'date': '2019-01-09T10:37:54', 'contents': '\\uce68 \\ud758\\ub9ac\\uba74 \\uc548\\ub3fc....\\n\\ud600 \\uc9d1\\uc5b4\\ub123\\uc73c\\uc138\\uc694 \\uc21c\\ubb34\\uc528\\u2764\\ufe0f\\n\\uadf8\\ub098\\uc800\\u

800 번째 데이터
{'find_tag': 'seoul', 'id': 'BsaCLtXD9os', 'username': 'hollygallery_', 'date': '2019-01-09T08:16:55', 'contents': 'designers \\nJames Holly\\n\\uc81c\\uc784\\uc2a4 \\ud640\\ub9ac\\nContemporary Art\\n#fineart\\n#oilpainting #digitalart #photographer #hollygallery #oilpainting #film #fashion #fineart #design #drawing #digitalart #sketch #sculpture #songwriter #art #artist #artgallery #contemporary #contemporaryart #modern #music #mueseum #masterpiece #illustration #installationart #internationalart #miami #goldcoast #london #paris #tokyo #seoul', 'hashtags': ['seoul', 'goldcoast', 'songwriter', 'music', 'illustration', 'film', 'design', 'fineart', 'modern', 'internationalart', 'artgallery', 'masterpiece', 'sculpture', 'contemporaryart', 'hollygallery', 'london', 'fashion', 'paris', 'digitalart', 'drawing', 'mueseum', 'contemporary', 'oilpainting', 'photographer', 'artist', 'tokyo', 'miami', 'sketch', 'art', 'installationart'], 'imagelinks': ['https://scontent-icn1-1.cdninsta

1600 번째 데이터
{'find_tag': 'seoul', 'id': 'BsaCsJZhb7N', 'username': 'azhar.farah', 'date': '2019-01-09T08:21:21', 'contents': 'Trout fish\\n@farahazhar902\\n#seoul', 'hashtags': ['seoul'], 'imagelinks': ['https://scontent-icn1-1.cdninstagram.com/vp/a91fd4be17168b378b9501a9d0f61c3b/5CD710AD/t51.2885-15/e35/47584820_352088655343492_3399466915480673759_n.jpg?_nc_ht=scontent-icn1-1.cdninstagram.com'], 'likes': '19 ', 'comments': ' 1 '}
------------------
1700 번째 데이터
{'find_tag': 'seoul', 'id': 'BsLjwZHB2UR', 'username': 'dr.mulligan1', 'date': 0, 'contents': [], 'hashtags': ['seoul', '고양이', '필드', '골프존', '한국', 'tour', '인천', '문의', 'klpga', '여행', '크리스마스', '투어', '골프스타그램', '해외여행', '환영', '부천', '서울', 'shots', '텍스트', 'korea', '다이렉트', '리조트', 'ㅋㅋㅋ', '경기도', 'kpga', '반려견', '디엠', '영상', '인싸', 'sns'], 'imagelinks': [], 'likes': 0, 'comments': 0}
------------------


MissingSchema: Invalid URL '': No schema supplied. Perhaps you meant http://?

In [24]:
len(json_list)

1711

In [25]:
## 생성한 데이터를 파일 형태로 저장합니다.

def save_jsonlist_json(myjson):
    file_name = my_tag + '_rawdata.json'
    with open(file_name, "w") as file:
        for line in myjson:
            file.write(str(line) + ',\n')

save_jsonlist_json(json_list)

In [26]:
## 앞으로 해야할 일

## 1. Like 와 Comment 형태를 1,000 등의 string 에서 1000 의 int 로 변환 저장
## 2. Unicode 형태의 데이터를 읽을 수 있는 데이터로 변환 -> 이모티콘을 출력하려 할 경우 멈춤
##    이모티콘을 제외하고 변환하여 사용하는 것이 맞을 것으로 보임
##    이와 관련해서는 ignore 옵션을 사용할 수 있을 것으로 보임

## !. CSV 로 저장된 데이터를 가져오니, 3400개가 1700개로 줄어들어 보여진다. 왜일까?
##    정확하게 절반으로 줄어든 것으로 보아 불러오는 코드에서 문제가 있는것 아닐까?

## !. time.sleep 을 주지 않고 크롤링 하니 300번째 정도에서 104 error 가 발생하였다
##    밴인지는 정확하게 모르겠지만, 몇번 더 확인해볼 필요가 있다. 0.1 sleep 에서는 정상작동 하였다.
##    여기서 점차 sleep time 을 줄여가며 속도를 올릴 필요가 있다